# Relationship Between GDP and Rental Prices

## Comparisons
Looking at Virginia cities, compare Virginia rental prices averaged by year to Virginia GDP.

## Method
1. Import Rent data
1. Create Rent DataFrame
1. Create GDP DataFrame from csv data and apply inflation method
1. Add base year GDP data into the Rent DataFrame
1. Run visualizations to find any trends in the data

In [1]:
# import needed libraries
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import common_functions

pd.set_option('display.max_rows', None)

pd.set_option('display.max_columns', None)

AttributeError: 'DataFrame' object has no attribute 'name'

## Read in the Zillow Yearly Rental Data 

In [ ]:
# load the zillow rental csv into a dataframe
richmondRent = pd.read_csv(r'../DataSet/RichmondRent.csv',header= 0,encoding = "ISO-8859-1")
vaBeachRent = pd.read_csv(r'../DataSet/VABeachRent.csv',header= 0,encoding = "ISO-8859-1")

In [ ]:
richmondRent.head()

In [ ]:
# delete blank first row
richmondRent.drop(richmondRent.index[0], axis = 0, inplace = True)
richmondRent.reset_index(drop = True, inplace = True)
richmondRent.head()

In [ ]:
vaBeachRent.head()

In [ ]:
# delete blank first row
vaBeachRent.drop(vaBeachRent.index[0], axis = 0, inplace = True)
vaBeachRent.reset_index(drop = True, inplace = True)
vaBeachRent.head()

In [ ]:
# now to concatenate the two data frames together

vaRent = pd.concat([richmondRent, vaBeachRent], ignore_index = True)
vaRent

In [ ]:
vaRent.dtypes

In [ ]:
for i in vaRent.index:
    vaRent.loc[i, 'median_rent_price'] = float(re.sub('[\D\s+]', '', vaRent.loc[i, 'median_rent_price']))
    vaRent.loc[i, 'mean_rent_price'] = float(re.sub('[\D\s+]', '', vaRent.loc[i, 'mean_rent_price']))
vaRent

In [ ]:
vaRent.dtypes

## Read in state GDP data

In [ ]:
gdpByState = pd.read_csv(r'../DataSet/yearlyGDPbyState.csv')
gdpByState.head(5)

In [ ]:
# drop old index column and reset index
gdpByState = gdpByState.drop('Unnamed: 0', axis = 1)

In [ ]:
gdpByState.head(5)

In [ ]:
gdpByState.dtypes

#### Now to inflate the dollars to base year 2019 dollars 

In [ ]:
gdpByState = common_functions.inflation_adjust(gdpByState)

In [ ]:
gdpByState.head()

#### DataFrame with GDP  for Virginia by year from 2014 - 2019

In [ ]:
gdpVA = gdpByState[(gdpByState['state'] == 'Virginia') & (gdpByState['year'] >= 2006) & (gdpByState['year'] <= 2017)]
gdpVA.reset_index(drop = True, inplace = True)
gdpVA

### Now to add GDP data to the various Rental DataFrames

#### stateRental DataFrame

In [ ]:
# add inflation adjusted GDP for Virginia to a dictionary with years as the key
gdp_dict = {}
for i in gdpVA.index:
    gdp_dict[gdpVA.loc[i, 'year']] = gdpVA.loc[i, 'inf-adjusted']
gdp_dict

In [ ]:
# add GDP to vaRent

for i in vaRent.index:
    vaRent.loc[i, 'Inf-Adj GDP'] = gdp_dict[vaRent.loc[i, 'yr']]

vaRent

## Data Visualizations

#### First, let's take a look at the scatter plot (no time-series) of the relationship between mean rent price and state GDP

In [ ]:
richmondVA = vaRent[vaRent['location_name'] == 'Richmond, VA']
vaBeachVA = vaRent[vaRent['location_name'] == 'Virginia Beach, VA']
richmondVA.head()

In [ ]:
vaBeachVA.head()

In [ ]:
plt.scatter(richmondVA['Inf-Adj GDP'],
            richmondVA['mean_rent_price'],
            color='red',
            label='Richmond')
plt.scatter(vaBeachVA['Inf-Adj GDP'],
            vaBeachVA['mean_rent_price'],
            color='blue',
            label='Virginia Beach')
plt.xlabel('Inflation Adjusted GDP of Virginia')
plt.ylabel('Mean Rent Price')
plt.legend()
plt.title('Inflation Adjusted GDP of Virginia Against the Mean Rent Price of Two Virginia Cities')
plt.show()
plt.savefig("GDPvsRent.jpg")

#### There is a slight positive trend between rent prices and GDP, indicating when GDP increases that rent increases as well. What is also interesting is the very clear divide in rent prices between Richmond and Virginia Beach.

#### Now let's look at the data as time series: 

In [ ]:
fig, ax1 = plt.subplots()

color = 'tab:red'
ax1.set_xlabel('Year')
ax1.set_ylabel('Mean Rent Price', color=color)
ax1.plot(richmondVA['yr'], richmondVA['mean_rent_price'], color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color2 = 'tab:blue'
ax2.set_ylabel('Inflation Adjusted GDP', color=color2)  # we already handled the x-label with ax1
ax2.plot(richmondVA['yr'], richmondVA['Inf-Adj GDP'], color=color2)
ax2.tick_params(axis='y', labelcolor=color2)

fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.title('Time Series Mean Rent Price and GDP for Richmond, VA')
plt.show()

#### Interestingly, it seems that rent increase/decrease lags behind the GDP. For example, 2010 saw a sharp decline in GDP but only a slight decline in rent. In 2011, there was a sharp decline in rent, but an increase in GDP.

In [ ]:
fig, ax1 = plt.subplots()

color = 'tab:red'
ax1.set_xlabel('Year')
ax1.set_ylabel('Mean Rent Price', color=color)
ax1.plot(vaBeachVA['yr'], vaBeachVA['mean_rent_price'], color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color2 = 'tab:blue'
ax2.set_ylabel('Inflation Adjusted GDP', color=color2)  # we already handled the x-label with ax1
ax2.plot(vaBeachVA['yr'], vaBeachVA['Inf-Adj GDP'], color=color2)
ax2.tick_params(axis='y', labelcolor=color2)

fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.title('Time Series Mean Rent Price and GDP for Virginia Beach, VA')
plt.show()

#### Although more chaotic at first glance, there still seems to be some lag in rent changes when GDP increase/decreases.